In [1]:
import pandas as pd
import numpy
import torch 
import numpy as np
from torch.utils.data import Dataset, DataLoader
import sys
from vis_utils import * 
sys.path.append("..")
from utils import pad_collate
from dataloader_comma import CommaDataset
from dataloader_nuscenes import NUScenesDataset
from collections import Counter
from model import VTN
import matplotlib.pyplot as plt 
from PIL import Image
import glob
import os
from utils import * 
import re
import plotly.graph_objects as go


using concept features


In [2]:
concs = []
gt = []
for j, batch in enumerate(dataloader_comma):
    _, image_array, vego, angle, distance, m_lens, i_lens, s_lens, a_lens, d_lens = batch
    img = image_array
    img, angle, distance, vego = img.to(f'cuda:{gpu_num}'), angle.to(f'cuda:{gpu_num}'), distance.to(f'cuda:{gpu_num}'), vego.to(f'cuda:{gpu_num}')
    (logits, attns), concepts = model(img, angle, distance, vego)
    top5_indices = torch.tensor(concepts.squeeze()).topk(5).indices
    s = img.shape#[batch_size, seq_len, h,w,c]
    angle, distance, vego = angle.to("cpu"), distance.to("cpu"), vego.to("cpu")
    logits = logits.detach().cpu().to("cpu")
    concepts = concepts.detach().cpu().to("cpu")

    att = attns[0][:,:,0:concepts.shape[1]].detach()
    seq_len = att.shape[2]
    atten = att
    alignment_array = get_aligned_attention(atten.squeeze().cpu(), seq_len)
    speed_graph = alignment_array.sum(axis=0)[8:-8]
    speed_graph_0 = moving_average(speed_graph, 5)

    att = attns[1][:,:,0:concepts.shape[1]].detach()
    seq_len = att.shape[2]
    atten = att
    alignment_array = get_aligned_attention(atten.squeeze().cpu(), seq_len)
    speed_graph = alignment_array.sum(axis=0)[8:-8]
    speed_graph_1 = moving_average(speed_graph, 5)
    concs.extend(top5_indices.squeeze().cpu().tolist())
    gt.extend(angle.squeeze().cpu().tolist())

/tmp/ipykernel_2344793/2670292364.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  top5_indices = torch.tensor(concepts.squeeze()).topk(5).indices


In [3]:
df = pd.DataFrame()
df['gt']  = gt
df['concepts'] = concs
df_exploded = df.explode('concepts')
df_exploded['gt_class'] = df_exploded["gt"].apply(lambda x: "<10" if x < 10 else (">10, <30" if x < 30 and x > 10 else (">30, <50") if x < 50 and x > 30 else ">50"))
grouped = df_exploded.groupby(by="gt_class")['concepts'].apply(list)

In [4]:
all_scens = []
for i in range(len(grouped)): 
    count_dict = Counter(grouped.iloc[i])
    top_5 = count_dict.most_common(10)
    scens = []
    for elem in top_5:
        if elem[0] == 131: continue
        scens.append((scenarios[elem[0]], elem[1]))
    all_scens.append(scens)



In [5]:
conc_df = pd.DataFrame(grouped)
conc_df['all_scens'] = all_scens
conc_df['len'] = conc_df['concepts'].apply(lambda x: len(x))

In [6]:
conc_df

,concepts,all_scens,len
gt_class,,,
<10,"[13, 131, 39, 8, 4, 131, 45, 118, 39, 2, 131, ...",[(a photo of a car in close proximity on the r...,3620
">10, <30","[131, 38, 63, 39, 12, 131, 38, 63, 12, 136, 13...",[(a photo of a car in close proximity on the r...,7885
">30, <50","[131, 39, 13, 268, 4, 131, 13, 132, 38, 39, 13...",[(a photo of driving on a highway with streetl...,5890
>50,"[131, 13, 63, 39, 4, 131, 13, 63, 39, 38, 131,...",[(a photo of driving on a highway with streetl...,12605


In [7]:
colors = [
    'rgba(228, 122, 122, 0.5)',  # Light Red
    'rgba(249, 173, 144, 0.5)',  # Light Orange
    'rgba(252, 216, 144, 0.5)',  # Light Yellow
    'rgba(211, 229, 157, 0.5)',  # Light Green
    'rgba(161, 218, 180, 0.5)',  # Light Teal
    'rgba(138, 202, 235, 0.5)',  # Light Blue
    'rgba(164, 156, 207, 0.5)',  # Light Purple
    'rgba(212, 161, 199, 0.5)',  # Light Pink
    'rgba(223, 223, 223, 0.5)',  # Light Gray
    'rgba(180, 180, 180, 0.5)'   # Light Dark Gray
]

In [79]:
import plotly.io as pio

# Create a sample scatter plot


# Define the nodes and links of the Sankey diagram
nodes = [
    {'label': '<b>Distance >50</b>', 'color': "white"}
]

links = []
leng = conc_df.iloc[-1].len
for i, elem in enumerate(conc_df.iloc[-1].all_scens):
    label = "<b>" + elem[0].replace("a photo of", "").replace("driving on", "").replace("a highway with", "").replace("a highway", "") + "</b>"
    nodes.append({'label': label, 'color': colors[i+1]})
    res = {'source': 0, 'target': i+1, 'value': elem[1]/leng, 'color': colors[i+1]}
    links.append(res)


# Create the Sankey diagram figure
fig = go.Figure(data=[go.Sankey(
    node=dict(
        label=[node['label'] for node in nodes],
        color=[node['color'] for node in nodes]
    ),
    link=dict(
        source=[link['source'] for link in links],
        target=[link['target'] for link in links],
        value=[link['value'] for link in links],
        color=[link['color'] for link in links]
    )
)])

# Customize the layout of the Sankey diagram
fig.update_layout(
    title_text='',
    width=1200,  # Set the width of the plot
    height=500,  # Set the height of the plot
    font=dict(size=20, color='black'),
    plot_bgcolor='white',
    margin=dict(
        autoexpand=True,
        l=0,
        r=0,
        t=0,
        b=0
    )
)

# Display the Sankey diagram
fig.show()
pio.write_image(fig, 'dist50.pdf')


In [80]:
# Define the nodes and links of the Sankey diagram
nodes = [
    {'label': '<b>Distance <10</b>', 'color': "white"}
]

links = []
leng = conc_df.iloc[0].len
for i, elem in enumerate(conc_df.iloc[0].all_scens):
    label = "<b>" +  elem[0].replace("a photo of", "").replace("driving on", "").replace("a highway with", "").replace("a street while", "").replace("a street with", "") + "</b>"
    nodes.append({'label':label, 'color': colors[i+1]})
    res = {'source': 0, 'target': i+1, 'value': elem[1]/leng, 'color': colors[i+1]}
    links.append(res)

# Create the Sankey diagram figure
fig = go.Figure(data=[go.Sankey(
    node=dict(
        label=[node['label'] for node in nodes],
        color=[node['color'] for node in nodes]
    ),
    link=dict(
        source=[link['source'] for link in links],
        target=[link['target'] for link in links],
        value=[link['value'] for link in links],
        color=[link['color'] for link in links]
    )
)])

# Customize the layout of the Sankey diagram
fig.update_layout(
    title_text='',
    width=1200,  # Set the width of the plot
    height=500,  # Set the height of the plot
    font=dict(size=20, color='black'),
    margin=dict(
        autoexpand=True,
        l=0,
        r=0,
        t=0,
        b=0
    ),
    plot_bgcolor='white'
)

# Display the Sankey diagram
fig.show()
pio.write_image(fig, 'dist10.pdf')
